In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
dfTrain = pd.read_csv('../../../DataSet/trip_train.csv', 
                      parse_dates=['start_date', 'end_date'], 
                      infer_datetime_format=True)

dfTest = pd.read_csv('../../../DataSet/trip_test.csv', 
                     parse_dates=['start_date', 'end_date'], 
                     infer_datetime_format=True)

In [3]:
# Convierto a los SUSCRIBER en un 0
# Convierto a los CUSTOMER en un 1
subscriptionTypes = dfTrain.subscription_type.unique()
print subscriptionTypes

# Reemplazo por 0 y 1
dfTrain.subscription_type = dfTrain.subscription_type.astype('category', categories=subscriptionTypes).cat.codes
dfTest.subscription_type = dfTest.subscription_type.astype('category', categories=subscriptionTypes).cat.codes

['Subscriber' 'Customer']


In [4]:
# GENERO TARGET, TRAIN, TEST Y TESTIDS (testids para el output)
target = dfTrain.duration
testIds = dfTest['id']


trainDateData = {'start_month':dfTrain.start_date.dt.month,
                 'start_dayOfYear':dfTrain.start_date.dt.dayofyear,
                 'start_dayOfWeek':dfTrain.start_date.dt.dayofweek, 
                 'start_hourOfDay':dfTrain.start_date.dt.hour}

testDateData = {'start_month':dfTest.start_date.dt.month,
                'start_dayOfYear':dfTest.start_date.dt.dayofyear,
                'start_dayOfWeek':dfTest.start_date.dt.dayofweek, 
                'start_hourOfDay':dfTest.start_date.dt.hour}

# Agrego columnas con la informacion de fechas a los dataframes
dfTrain = dfTrain[['start_station_id', 'subscription_type']].join(pd.DataFrame(trainDateData), how='outer')
dfTest = dfTest[['start_station_id', 'subscription_type']].join(pd.DataFrame(testDateData), how='outer')

In [5]:
rf = RandomForestRegressor(n_estimators=20, n_jobs=-1)

print("Volcando puntos...")
rf.fit(dfTrain, target)

print("Prediciendo...")
predictions = rf.predict(dfTest)



Volcando puntos...
Prediciendo...


In [6]:
dfTrips = pd.read_csv('../../../DataSet/trip.csv')[['id', 'duration']]
dfTrips.rename(columns={'duration':'real_duration'}, inplace=True)
dfValidation = pd.DataFrame({'id':testIds, 'pred_duration':predictions})
dfValidation = dfValidation.merge(dfTrips, how='inner', on='id')
dfValidation.head(10)

,id,pred_duration,real_duration
0,504737,6392.368512,2839
1,530846,331.150000,519
2,813140,568.205732,176
3,897674,599.506548,502
4,322830,612.496944,677
5,487841,558.146012,341
6,677808,432.547540,742
7,704449,689.479167,345
8,833587,486.452857,357
9,420411,494.576667,705


In [7]:
mean_squared_error(dfValidation.real_duration, dfValidation.pred_duration)

51895150.447191782